In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.power import TTestIndPower
import re

In [ ]:
station = pd.read_csv("./094/station.csv", sep='\t', engine="python")
patient = pd.read_csv("./094/patient.csv", sep='\t', engine="python")
observation = pd.read_csv("./094/observation.csv", sep='\t', engine="python")

## 2.1 A

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = observation['oximetry'].values
x = observation.drop(columns=['oximetry'], axis=1).values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# train_df = X_train.copy()
# train_df["oximetry"] = y_train
# 
# test_df = X_test.copy()
# test_df["oximetry"] = y_test
# 
# train_df.to_csv("train_raw.csv", index=False)
# test_df.to_csv("test_raw.csv", index=False)

## 2.1 B

In [ ]:
train_df = pd.read_csv("train_raw.csv")

In [ ]:
X_train.dtypes

In [ ]:
X_train.isnull().sum().sum()

In [ ]:
X_train.duplicated().sum()

## 2.1 C

In [ ]:
from scipy.stats import skew

skews = skew(x, axis=0)

skew_df = pd.DataFrame({
    'Feature': x.columns,
    'Skewness': skews
})

skew_df

Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler 

scaler = MinMaxScaler()

X_mm = scaler.fit_transform(X_train) 

X_mm

In [ ]:
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()

X_std = scaler.fit_transform(X_train) 

X_std

In [ ]:
import numpy as np

print("StandardScaler:")
print("Среднее =", np.mean(X_train, axis=0).round(3))
print("Ст. отклонение =", np.std(X_train, axis=0).round(3))

print("\nMinMaxScaler:")
print("Минимум =", np.min(X_train, axis=0).round(3))
print("Максимум =", np.max(X_train, axis=0).round(3))

In [ ]:
import numpy as np

print("StandardScaler:")
print("Среднее =", np.mean(X_std, axis=0).round(3))
print("Ст. отклонение =", np.std(X_std, axis=0).round(3))

print("\nMinMaxScaler:")
print("Минимум =", np.min(X_mm, axis=0).round(3))
print("Максимум =", np.max(X_mm, axis=0).round(3))

Transformers

In [ ]:
from sklearn.preprocessing import PowerTransformer
from matplotlib import pyplot

pyplot.hist(X_train[0], bins=10)

power = PowerTransformer(method='yeo-johnson', standardize=True) 
X_pt = power.fit_transform(X_train)

pyplot.hist(X_pt[0], bins=10) 

In [ ]:
from sklearn.preprocessing import QuantileTransformer
from matplotlib import pyplot

pyplot.hist(X_train[0], bins=10)

power = QuantileTransformer(output_distribution='normal', random_state=42)
X_qt = power.fit_transform(X_train)

pyplot.hist(X_qt[0], bins=10)

## 2.2 A

### Корреляционный анализ (линейная связь)

In [ ]:
corr = train_df.corr(numeric_only=False) 
# corr['oximetry'].sort_values()
corr_sorted = corr[['oximetry']].reindex(
    corr['oximetry'].abs().sort_values(ascending=False).index
)
corr_sorted

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(corr_sorted, annot=True, cmap='coolwarm')
plt.show()

### ANOVA F-test (feature selection test)

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

X = train_df.drop(columns=['oximetry'])
y = train_df['oximetry']

selector = SelectKBest(score_func=f_regression, k=2)
X_new = selector.fit_transform(X, y)

print("Shape после отбора:", X_new.shape)

idx = selector.get_support(indices=True)
print("Выбранные признаки:", list(X.columns[idx]))

print("F-значения выбранных признаков:",
      selector.scores_[idx])

In [ ]:
from sklearn.feature_selection import f_regression

X = train_df.drop(columns=['oximetry'])
y = train_df['oximetry']

f_values, p_values = f_regression(X, y)

anova_results = pd.DataFrame({
    'Feature': X.columns,
    'F_value': f_values,
    'p_value': p_values
}).sort_values(by='F_value', ascending=False)

anova_results

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

X = train_df.drop(columns=['oximetry'])
y = train_df['oximetry']

selector = SelectKBest(score_func=f_regression, k='all')
X_new = selector.fit_transform(X, y)

idx = selector.get_support(indices=True)
selected_features = X.columns[idx]

anova_results = pd.DataFrame({
    'Feature': selected_features,
    'F_value': selector.scores_[idx],
    'p_value': selector.pvalues_[idx]
}).sort_values(by='F_value', ascending=False)

print("Shape после отбора:", X_new.shape)
print("Выбранные признаки:", list(selected_features))
print(anova_results)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=anova_results, x='F_value', y='Feature')
plt.xscale('log')
plt.title('ANOVA F-test (log scale)')
plt.show()

### Mutual Information (нелинейная зависимость)

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
import pandas as pd
import matplotlib.pyplot as plt

selector = SelectKBest(score_func=mutual_info_regression, k=4)
selector.fit(X, y)

scores = pd.Series(abs(selector.scores_), index=X.columns).sort_values()

plt.figure(figsize=(10,5))
scores.plot(kind='barh')
plt.show()

print(scores.sort_values(ascending=False))

In [ ]:
corr_part = corr_sorted['oximetry'].abs().rename('Pearson_|r|')
anova_part = anova_results.set_index('Feature')['F_value'].rename('ANOVA_F')
mi_part = scores.rename('Mutual_Info')

compare_simple = pd.concat([corr_part, anova_part, mi_part], axis=1)
compare_simple = compare_simple.drop(index='oximetry', errors='ignore')
display(compare_simple)

# === 1️⃣ Нормализация в диапазоне [0,1] для каждой метрики ===
norm = compare_simple.apply(lambda s: (s - s.min()) / (s.max() - s.min()), axis=0)

# === 2️⃣ Добавляем среднее значение для финального сравнения ===
norm['Mean_Score'] = norm.mean(axis=1)

# === 3️⃣ Сортируем по информативности ===
final_display = norm.sort_values('Mean_Score', ascending=False)

final_display.style.format('{:.6f}')

## 2.2 B

In [ ]:
ranked_features = final_display.sort_values('Mean_Score', ascending=False)

display(ranked_features.style.format('{:.6f}'))

plt.figure(figsize=(10,6))
sns.barplot(data=ranked_features.head(10).reset_index(), x='Mean_Score', y='index')
plt.title('Top 10 ')
plt.show()

## 2.2 C
### (C-1b) Zdôvodnenie rozhodnutí pri realizácii

Pri realizácii úlohy som sa rozhodol využiť tri komplementárne prístupy na hodnotenie informatívnosti atribútov:
**Pearsonova korelácia**, **ANOVA F-test** a **Mutual Information**.  
Tieto techniky reprezentujú tri rôzne pohľady na vzťah medzi vstupnými premennými a cieľovou premennou *oximetry*:

- **Pearsonova korelácia** umožňuje zachytiť **lineárnu závislosť** medzi atribútom a cieľovou premennou.
- **ANOVA F-test** zisťuje, či existujú **štatisticky významné rozdiely v priemeroch** medzi skupinami hodnôt a hodnotí lineárnu relevantnosť premenných.
- **Mutual Information** je **nelineárna metóda**, ktorá kvantifikuje množstvo informácie, ktorú jeden atribút poskytuje o inom, a teda odhaľuje aj zložitejšie vzťahy.

Výber týchto troch metód umožnil porovnať lineárne aj nelineárne súvislosti a získať
robustnejší prehľad o dôležitosti atribútov.

Všetky výsledky boli následne **normalizované pomocou min–max transformácie** do intervalu ⟨0, 1⟩,  
aby bolo možné spravodlivo porovnávať hodnoty z rôznych metód, ktoré majú odlišné jednotky a rozsahy.
Z týchto normalizovaných hodnôt bol vypočítaný **priemerný index informatívnosti (Mean Score)**,
ktorý vyjadruje celkovú dôležitosť atribútu naprieč metódami.

Týmto postupom bolo zabezpečené:
- porovnateľnosť výsledkov z rôznych štatistických metód,
- odstránenie vplyvu rozdielnych mierok hodnôt (napr. F-hodnoty vs. korelácia),
- transparentný a reprodukovateľný spôsob zoradenia atribútov podľa dôležitosti.

Takto spracované výsledky umožňujú jednoznačne určiť,
ktoré atribúty majú najväčší vplyv na predikciu *oximetry*
a sú vhodné pre ďalšiu tvorbu predikčných modelov.
